# Install libraries

# Imports

In [18]:
import pandas as pd
import numpy as np
from alpaca.data import CryptoHistoricalDataClient
from alpaca.data.requests import CryptoBarsRequest
from alpaca.data.timeframe import TimeFrame
import talib
from datetime import datetime, timedelta
import configparser

/venv/trading_bot/lib/python3.12/site-packages/pytorch_forecasting/models/base_model.py:27: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


# Load configuration

In [19]:
# Load configuration
config = configparser.ConfigParser()
config.read('default.config')

# Initialize Alpaca client
client = CryptoHistoricalDataClient(
    api_key=config['alpaca']['API_KEY'],
    secret_key=config['alpaca']['API_SECRET']
)

# Download data

In [20]:
def fetch_crypto_data():
    """
    Fetch historical crypto data from Alpaca at 5-minute intervals
    """
    # Initialize client (no keys required for crypto data)
    client = CryptoHistoricalDataClient()
    
    # Create the request with multiple symbols
    request_params = CryptoBarsRequest(
        symbol_or_symbols=["BTC/USD", "ETH/USD"],
        timeframe=TimeFrame.Minute,  # Changed to 5-minute intervals
        start=datetime(2023, 1, 1),
        end=datetime(2024, 1, 1)
    )
    
    # Get the data
    bars = client.get_crypto_bars(request_params)
    
    # Convert to DataFrame and also get individual symbol data
    df = bars.df
    btc_bars = bars["BTC/USD"]  # Access specific symbol data
    
    return df, btc_bars

def compute_indicators(df):
    """
    Compute technical indicators
    """
    # Since fetch_crypto_data returns a tuple, get just the dataframe
    df = df[0]
    
    # Calculate SMA
    df['SMA_20'] = talib.SMA(df['close'], timeperiod=20)
    df['SMA_50'] = talib.SMA(df['close'], timeperiod=50)
    
    # Calculate EMA
    df['EMA_20'] = talib.EMA(df['close'], timeperiod=20)
    
    # Calculate Bollinger Bands
    upper, middle, lower = talib.BBANDS(df['close'], timeperiod=20)
    df['BBL_20_2.0'] = lower
    df['BBM_20_2.0'] = middle
    df['BBU_20_2.0'] = upper
    
    # Calculate RSI
    df['RSI'] = talib.RSI(df['close'], timeperiod=14)
    
    # Calculate MACD
    macd, macd_signal, macd_hist = talib.MACD(df['close'], fastperiod=12, slowperiod=26, signalperiod=9)
    df['MACD'] = macd
    df['MACD_signal'] = macd_signal
    df['MACD_hist'] = macd_hist
    
    # Normalize indicators and close price using Min-Max scaling
    columns_to_normalize = ['close', 'SMA_20', 'SMA_50', 'EMA_20', 'BBL_20_2.0', 'BBM_20_2.0', 'BBU_20_2.0', 'RSI', 'MACD', 'MACD_signal', 'MACD_hist']
    for col in columns_to_normalize:
        df[f'{col}_normalized'] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())
    return df

In [5]:
# Fetch data
df = fetch_crypto_data()

# Compute indicators
df = compute_indicators(df)

# Display first few rows
print(df.head())

# Display last few rows
print(df.tail())


In [6]:
# Keep only the relevant columns
columns_to_keep = [
    'volume',
    'trade_count',
    'close_normalized',
    'SMA_20_normalized', 
    'SMA_50_normalized',
    'EMA_20_normalized',
    'BBL_20_2.0_normalized',
    'BBM_20_2.0_normalized', 
    'BBU_20_2.0_normalized',
    'RSI_normalized',
    'MACD_normalized',
    'MACD_signal_normalized',
    'MACD_hist_normalized'
]

df = df[columns_to_keep]

# Remove any rows with NaN values that may have been created during indicator calculation
df = df.dropna()

print("Shape after keeping only relevant columns:", df.shape)
print("\nFirst few rows of cleaned dataset:")
print(df.head())


In [7]:
df.info()

In [8]:
# Save the preprocessed DataFrame to a CSV file
df.to_csv('preprocessed_data.csv', index=True)
print("Preprocessed data saved to 'data/preprocessed_data.csv'")